Copyright © 2021-2023 HQS Quantum Simulations GmbH. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except
in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.

# Measuring qubits in roqoqo

This notebook is designed to demonstrate the use of measurements in roqoqo. We will look at several examples of measuring qubits, from single and multi-qubit registers. To learn about the effect of measurement, we will look at the state vectors before and after measurement. 

In [ ]:
:dep roqoqo = "1.2.5"
:dep roqoqo-quest = "0.9.1"
:dep num-complex = "0.4"

In [ ]:
extern crate num_complex;
extern crate roqoqo;
extern crate roqoqo_quest;

use roqoqo::{Circuit, operations,prelude::*};
use roqoqo_quest::Backend;
use std::collections::HashMap;

## Measuring a single qubit

Here we first prepare the qubit in a superposition state, 
\begin{equation}
|+ \rangle = \frac{1}{\sqrt{2}} \big ( |0 \rangle + |1 \rangle \big ).
\end{equation}
We look at the state after preparation, then do a measurement in the Z basis, and finally look again at the state after measurement. 

We see that the state after measurement has been projected into the state either $|0>$ or $|1>$, consistently with the measurement outcome. Running this code many times should result in a random distribution of 'True' and 'False' outcomes.

In [ ]:
let mut state_init = Circuit::new(); 
state_init += operations::Hadamard::new(0); //prepare |+> state

// write state before measuring to readout register 'psi_in'
let mut read_input = Circuit::new(); 
read_input += operations::DefinitionComplex::new("psi_in".to_string(), 2, true);
read_input += operations::PragmaGetStateVector::new("psi_in".to_string(), Some(Circuit::new()));


// measure qubit in Z basis and write result to classical register 'M1'
let mut meas_circ = Circuit::new(); 
meas_circ += operations::DefinitionBit::new("M1".to_string(), 1, true);
meas_circ += operations::MeasureQubit::new(0, "M1".to_string(), 0);


// write state after measuring to readout register 'psi_out'
let mut read_output = Circuit::new();
read_output += operations::DefinitionComplex::new("psi_out".to_string(), 2, true);
read_output += operations::PragmaGetStateVector::new("psi_out".to_string(), Some(Circuit::new()));

// put each step of the circuit together
let circuit = state_init + read_input + meas_circ + read_output;

// run the circuit and collect output
let mut backend = roqoqo_quest::Backend::new(1);
let result_of_run = backend.run_circuit(&circuit);
let (result_bit_registers, _result_float_registers, result_complex_registers)
= result_of_run.unwrap();

println!("Input state:{:?}",result_complex_registers["psi_in"]);
println!("Measurement result:{:?}",result_bit_registers["M1"]);
println!("State after measurement:{:?} \n",result_complex_registers["psi_out"]);


## Measuring a single qubit in the X basis

Instead of measuring in the Z basis, we can measure the qubit in the X basis by performing a Hadamard operator before the measurement. 

This time we see that the measurement result is always 'False', since we are measuring the $|+ \rangle$ state in the X basis, and it is an X eigenvector of the X operator. 

In [ ]:
let number_of_qubits: usize = 3;

let mut state_init = Circuit::new();
state_init += operations::PauliX::new(1);
state_init += operations::Hadamard::new(0);
state_init += operations::CNOT::new(0, 1);
state_init += operations::CNOT::new(0, 2);
state_init += operations::SGate::new(0);

// write state before measuring to readout register 'psi_in'
let mut read_input = Circuit::new();
read_input += operations::DefinitionComplex::new("psi_in".to_string(), 2usize.pow(number_of_qubits as u32),  true);
read_input += operations::PragmaGetStateVector::new("psi_in".to_string(), Some(Circuit::new()));

// measure qubits in Z basis and write result to classical register 'M1M2M3'
let mut meas_circ = Circuit::new();
meas_circ += operations::DefinitionBit::new("M1M2M3".to_string(), 3, true);
meas_circ += operations::MeasureQubit::new(0, "M1M2M3".to_string(), 0);
meas_circ += operations::MeasureQubit::new(1, "M1M2M3".to_string(), 1);
meas_circ += operations::MeasureQubit::new(2, "M1M2M3".to_string(), 2);

// write state after measuring to readout register 'psi_out'
let mut read_output = Circuit::new();
read_output += operations::DefinitionComplex::new("psi_out".to_string(), 2usize.pow(number_of_qubits as u32),true );
read_output += operations::PragmaGetStateVector::new("psi_out".to_string(), Some(Circuit::new()));

let mut circuit = state_init + read_input + meas_circ + read_output;

// run the circuit and collect output
let backend = roqoqo_quest::Backend::new(number_of_qubits);
let result_of_run = backend.run_circuit(&circuit);
let (result_bit_registers,_result_float_registers,result_complex_registers) 
= result_of_run.unwrap_or_default();

println!("Input state:{:?}",result_complex_registers["psi_in"]);
println!("Measurement result:{:?}",result_bit_registers["M1M2M3"]);
println!("State after measurement:{:?} \n",result_complex_registers["psi_out"]);

## Measuring a multi-qubit register

Here we first prepare a multi-qubit register and demonstrate how it is possible to measure the entire register. As an example we prepare the multi-qubit register in the state, 
\begin{equation}
|\psi \rangle = \frac{1}{\sqrt{2}} |010 \rangle + \frac{i}{\sqrt{2}} |101 \rangle.
\end{equation}

After preparation we read out the simulated state, before measurement. Next we measure each qubit of the state, and finally we readout out the post-measurement state. 

In [ ]:
let number_of_qubits: usize = 3;

let mut state_init = Circuit::new();
state_init += operations::PauliX::new(1);
state_init += operations::Hadamard::new(0);
state_init += operations::CNOT::new(0, 1);
state_init += operations::CNOT::new(0, 2);
state_init += operations::SGate::new(0);

// write state before measuring to readout register 'psi_in'
let mut read_input = Circuit::new();
read_input += operations::DefinitionComplex::new("psi_in".to_string(), 2usize.pow(number_of_qubits as u32),  true);
read_input += operations::PragmaGetStateVector::new("psi_in".to_string(), Some(Circuit::new()));

// measure qubit in Z basis and write result to classical register 'M1'
let mut meas_circ = Circuit::new();
meas_circ += operations::DefinitionBit::new("M1".to_string(), 1, true);
meas_circ += operations::MeasureQubit::new(0, "M1".to_string(), 0);

// write state after measuring to readout register 'psi_out'
let mut read_output = Circuit::new();
read_output += operations::DefinitionComplex::new("psi_out".to_string(), 2usize.pow(number_of_qubits as u32),true );
read_output += operations::PragmaGetStateVector::new("psi_out".to_string(), Some(Circuit::new()));

let circuit = state_init + read_input + meas_circ + read_output;

// run the circuit and collect output
let backend = roqoqo_quest::Backend::new(number_of_qubits);
let result_of_run = backend.run_circuit(&circuit);
let (result_bit_registers,_result_float_registers,result_complex_registers) 
= result_of_run.unwrap_or_default();

println!("Input state:{:?}",result_complex_registers["psi_in"]);
println!("Measurement result:{:?}",result_bit_registers["M1"]);
println!("State after measurement:{:?}",result_complex_registers["psi_out"]);